In [240]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from dummyPy import OneHotEncoder
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
import matplotlib.pyplot as plt
#
from xgboost import XGBClassifier


In [241]:
train=pd.read_csv('./data/train_users_2.csv', encoding='utf-8')
test=pd.read_csv('./data/test_users.csv', encoding='utf-8')
dataset=pd.read_csv('./data/train_users_2.csv', encoding='utf-8')
labels=train['country_destination']

In [242]:
df_all=pd.read_csv('./data/airbnb_data.csv', encoding='utf-8')

C:\Python\Anaconda3-5.2\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [257]:
for i in range(len(df_all['age_range'])):
    df_all['age_range'][i] = str(df_all['age_range'][i]).replace(']','').replace(',','').replace('(','')

C:\Python\Anaconda3-5.2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

In [244]:
df_all.drop(['Unnamed: 0','timelag','timestamp_first_active_temp','timestamp_first_active_d','date_account_created_d','timestamp_first_active','date_account_created','train_test'], axis=1, inplace=True)

In [245]:
df_all.columns

Index(['affiliate_channel', 'affiliate_provider', 'country_destination',
       'first_affiliate_tracked', 'first_browser', 'first_device_type', 'id',
       'language', 'signup_app', 'signup_flow', 'signup_method', 'booked',
       'Year', 'age_range', 'timelag_nu'],
      dtype='object')

In [246]:
encoder = OneHotEncoder(['affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'first_browser', 'first_device_type',
       'language', 'signup_app', 'signup_method',
       'booked', 'Year', 'age_range'])

In [247]:
encoder.fit(df_all)
df_combine=encoder.transform(df_all)

In [268]:
pd.set_option('display.max_rows', 1000)

In [249]:
df_combine.dtypes

affiliate_channel_api                     object
affiliate_channel_content                 object
affiliate_channel_direct                  object
affiliate_channel_other                   object
affiliate_channel_remarketing             object
affiliate_channel_sem-brand               object
affiliate_channel_sem-non-brand           object
affiliate_channel_seo                     object
affiliate_provider_baidu                  object
affiliate_provider_bing                   object
affiliate_provider_craigslist             object
affiliate_provider_daum                   object
affiliate_provider_direct                 object
affiliate_provider_email-marketing        object
affiliate_provider_facebook               object
affiliate_provider_facebook-open-graph    object
affiliate_provider_google                 object
affiliate_provider_gsp                    object
affiliate_provider_meetup                 object
affiliate_provider_naver                  object
affiliate_provider_o

In [250]:
df_combine

,affiliate_channel_api,affiliate_channel_content,affiliate_channel_direct,affiliate_channel_other,affiliate_channel_remarketing,affiliate_channel_sem-brand,affiliate_channel_sem-non-brand,affiliate_channel_seo,affiliate_provider_baidu,affiliate_provider_bing,...,"age_range_(0, 21]","age_range_(21, 31]","age_range_(31, 41]","age_range_(41, 51]","age_range_(51, 61]","age_range_(61, 71]","age_range_(71, 81]","age_range_(81, 91]","age_range_(91, 100]",timelag_nu
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,466
1,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,732
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,476
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,765
4,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,280
5,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [278]:
mapping= {'NDF':0, 'US':1, 'other':2, 'FR':3, 'CA':4, 'GB':5, 'ES':6, 'IT':7, 'PT':8, 'NL':9, 'DE':10, 'AU':11}

Y_train=df_combine[df_combine['id'].isin(train['id'])]
#Y_train=Y_train['country_destination'].map(mapping)
#df_combine=df_combine.drop(['country_destination'],axis=1)

X_train=df_combine[df_combine['id'].isin(train['id'])]
X_test=df_combine[df_combine['id'].isin(test['id'])]
X_test=X_test.drop(['id'],axis=1)
X_train=X_train.drop(['id'],axis=1)
Y_train=pd.DataFrame(Y_train)

X_train=X_train.astype('float32', copy=False)
X_test=X_test.astype('float32', copy=False)
Y_train=Y_train.astype('int32', copy=False)

ValueError: invalid literal for int() with base 10: 'gxn3p5htnn'

In [265]:
df_combine.rename(index=str, columns={'age_range_(0, 21]':'Age : 0~21', 'age_range_(21, 31]':'Age : 21~31', 'age_range_(31, 41]':'Age : 31~41',
       'age_range_(41, 51]':'Age : 41~51', 'age_range_(51, 61]':'Age : 51~61', 'age_range_(61, 71]':'Age : 61~71',
       'age_range_(71, 81]':'Age : 71~81', 'age_range_(81, 91]':'Age : 81~91', 'age_range_(91, 100]':'Age : 91~100'}, inplace=True)

In [262]:
df_combine.columns[-10] = 'Age : 0~21'
df_combine.columns[-9] = 'Age : 21~31'
df_combine.columns[-8] = 'Age : 31~41'
df_combine.columns[-7] = 'Age : 41~51'
df_combine.columns[-6] = 'Age : 51~61'
df_combine.columns[-5] = 'Age : 61~71'
df_combine.columns[-4] = 'Age : 71~81'
df_combine.columns[-3] = 'Age : 81~91'
df_combine.columns[-2] = 'Age : 91~100'


TypeError: Index does not support mutable operations

In [275]:
pd.DataFrame(df_combine.columns).to_csv('./data/check.csv')

In [193]:
#XGBOOST

In [267]:
import xgboost as xgb
import time
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=5)

dtrain = xgb.DMatrix(x_train,y_train)
dtest = xgb.DMatrix(x_test)

# specify parameters via map

param = {'max_depth': 9, 'eta': 0.1, 'silent': 1, 'objective': 'multi:softmax',"num_class": 12, 'subsample' : 0.5, 'colsample_bytree' : 0.5,
               'seed' : 1,
               'eval_metric' : "merror",
               'nthread' : 3}
num_round = 25
training_start = time.perf_counter()
bst = xgb.train(param, dtrain, num_round)
training_end = time.perf_counter()
# make prediction
prediction_start = time.perf_counter()
preds = bst.predict(dtest)
prediction_end = time.perf_counter()
nd_y_test=y_test.values.reshape(len(y_test.values))
acc_xgb = (preds == nd_y_test).sum().astype(float) / len(preds)*100
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

ValueError: feature_names may not contain [, ] or <

In [28]:
#Randomfrest

In [29]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
np.random.seed(0)

In [30]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=7)
clf = RandomForestClassifier(n_jobs=100, random_state=0)
training_start = time.perf_counter()
clf.fit(x_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds=clf.predict(x_test)
prediction_end = time.perf_counter()
nd_y_test=y_test.values.reshape(len(y_test.values))
acc_xgb = (preds == nd_y_test).sum().astype(float) / len(preds)*100
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
print("RF's prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

C:\Python\Anaconda3-5.2\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


XGBoost's prediction accuracy is: 57.34
Time consumed for training: 2.212
Time consumed for prediction: 0.28585 seconds


In [117]:
#Softmax

In [118]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=7)

encoder = OneHotEncoder(['country_destination'])
encoder.fit(y_train)
y_train=encoder.transform(y_train)
y_train=y_train.astype('int32', copy=False)

In [120]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

In [121]:
tf.reset_default_graph()
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility

# parameters
learning_rate = 0.0001
training_epochs = 15
batch_size = 100
nb_classes = 12
keep_prob = 0.7
# input place holders
X = tf.placeholder(tf.float32, [None, 161])
Y = tf.placeholder(tf.int32, [None, 12])

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[161, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.get_variable("W2", shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([128]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.get_variable("W3", shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.get_variable("W4", shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([128]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.get_variable("W5", shape=[128, 12],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([12]))
logits = tf.matmul(L4, W5) + b5
hypothesis = tf.nn.softmax(logits)


# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(y_train, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [122]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    feed_dict = {X: x_train, Y: y_train}
    sess.run(optimizer, feed_dict=feed_dict)
    if step % 100 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={X: x_train, Y: y_train})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(step, loss, acc))

Step:     0, 	 Loss: 2.533, 	 Acc: 8.51%
Step:   100, 	 Loss: 2.327, 	 Acc: 29.19%
Step:   200, 	 Loss: 2.326, 	 Acc: 29.25%
Step:   300, 	 Loss: 2.326, 	 Acc: 29.26%


KeyboardInterrupt: 

In [195]:
#제출용 파일생성
X_test=xgb.DMatrix(X_test)
result=bst.predict(X_test)
result=pd.DataFrame(result)
result=result[0]
inv_mapping= {0:'NDF', 1:'US', 2:'other', 3:'FR', 4:'CA', 5:'GB', 6:'ES', 7:'IT', 8:'PT', 9:'NL', 10:'DE', 11:'AU'}
result=result.map(inv_mapping)
result_test=test.drop(['date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser'], axis=1)
result_test=result_test.join(result)
result_test.columns=['id','country']
filename='Submission_{}.csv'.format(acc_xgb) #Accracy 변수 설정
result_test.to_csv(filename,index=False)